In [ ]:
#%%capture
!pip install musdb
!pip install keras
!pip install tensorflow
!pip install tensorflow-gpu

  Using cached https://files.pythonhosted.org/packages/57/bd/98ba16482f610bcfa7fcc212175dc0bbf11976e0bc69319b4204b6dc3aec/musdb-0.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/dc/b37d98b2532aadfbf941695d6663ff0b54103795223cea8c17bf91a55183/stempeg-0.1.8.tar.gz
  Using cached https://files.pythonhosted.org/packages/eb/f2/3cbbbf3b96fb9fa91582c438b574cff3f45b29c772f94c400e2c99ef5db9/SoundFile-0.10.3.post1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/1e/eda9fe07f752ced7afcef590e7d74390f0d9c9c0b7ff98317afbaa0697e3/pyaml-19.12.0-py2.py3-none-any.whl
Could not install packages due to an EnvironmentError: [Errno 13] Permissão negada: '/usr/lib/python2.7/site-packages/tqdm-4.42.1.dist-info'
Consider using the `--user` option or check the permissions.

  Using cached https:

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd
from google.colab import drive
%tensorflow_version 2.x

ModuleNotFoundError: No module named 'keras'

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path = '/content/drive/My Drive/Projeto Final/'

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load(self.path + 'data_samples/' + ID + '.npy')

            # Store class
            y[i] = np.load(self.path + 'vocal_samples/' + ID + '.npy')


        return X, y

1.15.0
/device:GPU:0
Num GPUs Available:  1


In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
range(10)

In [0]:
with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(Conv2D(16, (3,3), padding='same', input_shape=(513, 11, 1)))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.25))
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(64))
  model.add(LeakyReLU())
  model.add(Dropout(0.5))
  model.add(Dense(513, activation='relu'))
  model.compile(loss=keras.losses.mse, optimizer='adam', metrics=['mse'])

In [0]:
with tf.device('/device:GPU:0'):
  model.fit(np.abs(train_data_groups).reshape(target_groups.shape[0],513,11,1),np.abs(target_groups),batch_size = 32,epochs = 30)

In [0]:
a = model.predict(np.abs(train_data_groups[0:1*1293].reshape(1293,513,25,1)))
test = np.transpose(a)
test_list = []
for i in range(len(test)):
  test_list.append(np.roll(test[i],2))
test = np.array(test_list)

In [0]:
vocal_stft_list

In [0]:
result_stft = np.multiply(test,stft_list[0])
plt.pcolormesh(t, f, np.abs(result_stft))
xmin =0
xmax = 1
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])


In [0]:
plt.pcolormesh(t, f, np.abs(vocal_stft_list[0]))
xmin =0
xmax = 1
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])

In [0]:
audio1 = signal.istft(result_stft,fs =mus[0].rate)
audio1

In [0]:
ipd.Audio(audio1[1],rate=mus[0].rate)

In [0]:
ipd.Audio(mus[0].audio[:,0] + mus[0].audio[:,1],rate=mus[0].rate)